# Homework 0203 Corpus Statistics
## Kyle Walter

In [1]:
%%capture
#import the Packages needed to work with the text
import os # will be used to loop through an get all the .txt files
import pandas as pd #for working with data frame
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer # for vectorization
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import *

In [2]:
#added to stopword list punctuation that isn't helpful for the analysis
standardStop = stopwords.words('english')
addedStop = ['.',',',"'","!","-",'"']
for x in addedStop:
    standardStop.append(x)
print(standardStop) # this prints out the stopword list

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Reading Text from corpus

In [3]:
#Creates a list to hold the the text data, then it reads in the text file and splits the line by the tab seperating the sentiment score
#Next next it complies the data into a dictinary and converts to a data frame. Lastly printed the DF info to make sure the information is there

text = []
for item in os.listdir():
    if item.endswith('.txt'):
        with open(item, 'r') as file:
            t = file.readlines()
            for item in range(0,len(t)):
                content = t[item]
                content = content.split('\t')
                content2 = content[1].split('\n')
                dictionary = {'Sent':content2[0], 'text':content[0]}
                text.append(dictionary)
corpusdf = pd.DataFrame(text)
corpusdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Sent    3000 non-null   object
 1   text    3000 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB


In [4]:
corpusdf['Sent'].value_counts() #checked the balance of the data set.

0    1500
1    1500
Name: Sent, dtype: int64

# Reading in Text from csv

In [5]:
# Reads in and renames the columns from homework 1
csvdf = pd.read_csv('twitterSentiment.csv', header = None, encoding='latin-1')
csvdf.rename(columns={0:"Sent",1:"ID",2:"Date of Tweet", 3:"Ignore", 4:"User", 5:"text"}, inplace = True)
csvdf.info() # inspects the dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Sent           1600000 non-null  int64 
 1   ID             1600000 non-null  int64 
 2   Date of Tweet  1600000 non-null  object
 3   Ignore         1600000 non-null  object
 4   User           1600000 non-null  object
 5   text           1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [6]:
#csvdf.loc[csvdf['Sent']==4,'Sent']=1
#csvdf['Sent'].value_counts()


In [6]:
#csvdf = csvdf[['Sent','text']]
#csvdf.head()

,Sent,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [7]:
#frames = [corpusdf, csvdf]
#corpusdf = pd.concat(frames)
#corpusdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1603000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Sent    1603000 non-null  object
 1   text    1603000 non-null  object
dtypes: object(2)
memory usage: 36.7+ MB


In [6]:
del csvdf #free up memory by removing the prior frame no longer in use

# Tokenization, lemmatize, stopswords

In [7]:
corpusdf['text']=corpusdf['text'].str.lower() #converts all words to lower case
corpusdf['sentence']=corpusdf['text'].apply(lambda x: nltk.sent_tokenize(x)) # Tokenizes by sentence
corpusdf['nmbrSentences']=corpusdf['sentence'].apply(lambda x: len(x)) # counts the number of senteces in each corpus
np.mean(corpusdf['nmbrSentences']) #check the average length of a review

1.019

In [8]:
corpusdf['text'][:5]

0                             wow... loved this place.
1                                   crust is not good.
2            not tasty and the texture was just nasty.
3    stopped by during the late may bank holiday of...
4    the selection on the menu was great and so wer...
Name: text, dtype: object

In [9]:
TokenizedText = [nltk.wordpunct_tokenize(x) for x in corpusdf['text']] #tokenizes the text
TokenizedText[:5] #display tokenized output

[['wow', '...', 'loved', 'this', 'place', '.'],
 ['crust', 'is', 'not', 'good', '.'],
 ['not', 'tasty', 'and', 'the', 'texture', 'was', 'just', 'nasty', '.'],
 ['stopped',
  'by',
  'during',
  'the',
  'late',
  'may',
  'bank',
  'holiday',
  'off',
  'rick',
  'steve',
  'recommendation',
  'and',
  'loved',
  'it',
  '.'],
 ['the',
  'selection',
  'on',
  'the',
  'menu',
  'was',
  'great',
  'and',
  'so',
  'were',
  'the',
  'prices',
  '.']]

In [10]:
#Removes stop words and connects them back together as Tokens
cleanedText = []
for sentence in TokenizedText:
    cleanedText.append(" ".join([word for word in sentence if word not in standardStop]))
cleanedText[:5] #shows output without stopwords

['wow ... loved place',
 'crust good',
 'tasty texture nasty',
 'stopped late may bank holiday rick steve recommendation loved',
 'selection menu great prices']

In [11]:
TokenizedText = [nltk.wordpunct_tokenize(x) for x in cleanedText] #tokenizes the text
TokenizedText[:5] #display tokenized output

[['wow', '...', 'loved', 'place'],
 ['crust', 'good'],
 ['tasty', 'texture', 'nasty'],
 ['stopped',
  'late',
  'may',
  'bank',
  'holiday',
  'rick',
  'steve',
  'recommendation',
  'loved'],
 ['selection', 'menu', 'great', 'prices']]

In [12]:
lemmer = WordNetLemmatizer() #defines the Lemmantizer
lemmaText = [] #creates an empty list for the lemma
for sentence in TokenizedText:
    lemmaText.append(",".join([lemmer.lemmatize(word) for word in sentence]))
lemmaText[:10] #displays Lemma Output

['wow,...,loved,place',
 'crust,good',
 'tasty,texture,nasty',
 'stopped,late,may,bank,holiday,rick,steve,recommendation,loved',
 'selection,menu,great,price',
 'getting,angry,want,damn,pho',
 'honeslty,taste,fresh,.)',
 'potato,like,rubber,could,tell,made,ahead,time,kept,warmer',
 'fry,great',
 'great,touch']

In [13]:
stemmer = PorterStemmer() #defines Stemmer
stemmedText =[] #creates an empty list for the Stemmer
for sentence in TokenizedText:
    stemmedText.append(",".join([stemmer.stem(word) for word in sentence]))
stemmedText[:5] #shows stemmer output

['wow,...,love,place',
 'crust,good',
 'tasti,textur,nasti',
 'stop,late,may,bank,holiday,rick,steve,recommend,love',
 'select,menu,great,price']

In [14]:
#looking at number of Unique Tokens
from itertools import chain
Tokens = len(set(chain(*TokenizedText)))
lemma = len(set(chain(*lemmaText)))
stems = len(set(chain(*stemmedText)))
print("Number of tokens: ",Tokens)
print("Number of Lemma Tokens: ", lemma)
print("Number of Stemmed Tokens: ", stems)


Number of tokens:  5122
Number of Lemma Tokens:  61
Number of Stemmed Tokens:  61


In [15]:
#freeing up membory before moving on to vectorization
del stemmedText
del lemmaText
del TokenizedText
del cleanedText

# Vectorize Count Vectorization

In [16]:
#Define Tokenize for the countvectorization
def nltkTokenizer(text):
    tokenization = nltk.wordpunct_tokenize(text)
    return tokenization
#define the CountVectorizer Parameters, it will do the prior precessing of the text
#with correct inputs it will also properly take care of items suchs as bigram creation around stopwords
vectorizer=CountVectorizer(
    lowercase=True, #converts words to lower case
    analyzer ='word', #method of counting is at the word/token instead of character
    ngram_range=(1,3), #Tokens contain how many grams, in this case between 1 and 3 words
    stop_words=standardStop, #sets stopword list to us, in this case from nltk
    tokenizer=nltkTokenizer)
counts = vectorizer.fit_transform(corpusdf['text'])

In [17]:
#Creates a tuple of the words with their count frequencies
sum_words = counts.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [18]:
words_freq[:20] #returns 20 most common words

[('good', 230),
 ('great', 210),
 ('movie', 182),
 ('phone', 168),
 ('film', 163),
 ('one', 146),
 ('food', 126),
 ('like', 125),
 ('place', 114),
 ('time', 112),
 ('service', 108),
 ('bad', 103),
 ('really', 103),
 ('well', 92),
 ('(', 90),
 ('would', 87),
 ('best', 78),
 ('even', 77),
 ('ever', 76),
 ('also', 74)]

In [19]:
words_freq[-20:]

[('work lost', 1),
 ('lost regrettable', 1),
 ('regrettable script', 1),
 ('unfortunately virtue film', 1),
 ('virtue film production', 1),
 ('film production work', 1),
 ('production work lost', 1),
 ('work lost regrettable', 1),
 ('lost regrettable script', 1),
 ('word embarrassing', 1),
 ('exceptionally', 1),
 ('exceptionally bad', 1),
 ('insult one', 1),
 ('one intelligence', 1),
 ('intelligence huge', 1),
 ('huge waste', 1),
 ('insult one intelligence', 1),
 ('one intelligence huge', 1),
 ('intelligence huge waste', 1),
 ('huge waste money', 1)]

In [20]:
len(vectorizer.vocabulary_)

32728

In [21]:
#Look at Matrix
pd.DataFrame(counts.toarray(), columns = vectorizer.get_feature_names())

,!!,!!!,!!!!,!!!!!,!!!!!!,!!!!!!!!,!!!!!!!!!,!!!!!!!!!!!!!!,!!!!!!!!!!!!!!!!!!!,!!!!!!!!.,..., particularly, particularly good, torture, totally, totally believable, true, true masterpiece,, acting, acting especially
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Vectorize Term Frequency inverse document frequency

In [22]:
tfVectorizer = TfidfVectorizer(
    lowercase=True, #converts words to lower case
    analyzer ='word', #method of counting is at the word/token instead of character
    ngram_range=(1,3), #Tokens contain how many grams, in this case between 1 and 3 words
    stop_words=standardStop, #sets stopword list to us, in this case from nltk
    tokenizer=nltkTokenizer)
tfidf = tfVectorizer.fit_transform(corpusdf['text'])

In [23]:
tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
len(tfVectorizer.vocabulary_)

32728

# Model Testing

In [25]:
#without negation stop words
nbModel = MultinomialNB()
cv = 10 #set up 10 fold cross validation

#Set up dependent and independent Variables
countX = counts
tfidfX = tfidf
y = corpusdf['Sent']

In [26]:
YpredCount = cross_val_predict(nbModel, countX, y, cv=cv)
print(metrics.classification_report(y, YpredCount))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81      1500
           1       0.82      0.81      0.81      1500

    accuracy                           0.81      3000
   macro avg       0.81      0.81      0.81      3000
weighted avg       0.81      0.81      0.81      3000



In [27]:
YpredTfidf = cross_val_predict(nbModel, tfidfX, y, cv=cv)
print(metrics.classification_report(y, YpredTfidf))

              precision    recall  f1-score   support

           0       0.83      0.81      0.82      1500
           1       0.81      0.83      0.82      1500

    accuracy                           0.82      3000
   macro avg       0.82      0.82      0.82      3000
weighted avg       0.82      0.82      0.82      3000



In [28]:
negations =['no', 'nor', 'not', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn']
for x in negations:
    standardStop.remove(x)
print(standardStop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

In [29]:
# with Negation Stop Words
counts = vectorizer.fit_transform(corpusdf['text'])
tfidf = tfVectorizer.fit_transform(corpusdf['text'])
#Set up dependent and independent Variables
countX = counts
tfidfX = tfidf
y = corpusdf['Sent']

In [30]:
sum_words = counts.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [31]:
words_freq[:20] #returns 20 most common words

[('not', 306),
 ('good', 230),
 ('great', 210),
 ('movie', 182),
 ('phone', 168),
 ('film', 163),
 ('one', 146),
 ('food', 126),
 ('like', 125),
 ('place', 114),
 ('time', 112),
 ('service', 108),
 ('bad', 103),
 ('really', 103),
 ('well', 92),
 ('(', 90),
 ('would', 87),
 ('no', 83),
 ('best', 78),
 ('even', 77)]

In [32]:
words_freq[-20:] #20 least frequent words

[('work lost', 1),
 ('lost regrettable', 1),
 ('regrettable script', 1),
 ('unfortunately virtue film', 1),
 ('virtue film production', 1),
 ('film production work', 1),
 ('production work lost', 1),
 ('work lost regrettable', 1),
 ('lost regrettable script', 1),
 ('word embarrassing', 1),
 ('exceptionally', 1),
 ('exceptionally bad', 1),
 ('insult one', 1),
 ('one intelligence', 1),
 ('intelligence huge', 1),
 ('huge waste', 1),
 ('insult one intelligence', 1),
 ('one intelligence huge', 1),
 ('intelligence huge waste', 1),
 ('huge waste money', 1)]

In [33]:
#Prediction with negation words left in for count vectorizer
YpredCount = cross_val_predict(nbModel, countX, y, cv=cv)
print(metrics.classification_report(y, YpredCount))

              precision    recall  f1-score   support

           0       0.82      0.85      0.83      1500
           1       0.84      0.82      0.83      1500

    accuracy                           0.83      3000
   macro avg       0.83      0.83      0.83      3000
weighted avg       0.83      0.83      0.83      3000



In [34]:
#prediction with Negation words left inf for tfidf vectorizer
YpredTfidf = cross_val_predict(nbModel, tfidfX, y, cv=cv)
print(metrics.classification_report(y, YpredTfidf))

              precision    recall  f1-score   support

           0       0.84      0.83      0.84      1500
           1       0.83      0.84      0.84      1500

    accuracy                           0.84      3000
   macro avg       0.84      0.84      0.84      3000
weighted avg       0.84      0.84      0.84      3000

